In [ ]:
import requests
import json
from datetime import datetime
import pyupbit
from anthropic import Anthropic
from typing import Dict, List, Any

class DocumentAnalysisAgent:
    """문서 분석 에이전트"""
    def __init__(self, api_key: str):
        self.claude = Anthropic(api_key=api_key)
        self.upstage_api_key = "up_IANAGmcf5Z0MK40wgyiNBYJAlFMTe"

    def parse_document(self, file_path: str) -> str:
        """Document Parser API로 PDF 텍스트 추출"""
        url = "https://api.upstage.ai/v1/document-ai/document-parse"
        headers = {"Authorization": f"Bearer {self.upstage_api_key}"}
        files = {"document": open(file_path, "rb")}
        data = {
            "ocr": "force",
            "coordinates": True,
            "chart_recognition": True,
            "output_formats": "['text']",
            "base64_encoding": "['table']",
            "model": "document-parse"
        }
        response = requests.post(url, headers=headers, files=files, data=data)
        if response.status_code == 200:
            return response.json()['content']['text']
        return None

    def analyze_document(self, text: str, market_data: Dict) -> Dict:
        """Claude를 사용하여 문서 분석"""
        prompt = f"""
        다음은 비트코인 관련 문서와 현재 시장 데이터입니다. 이를 분석하여 매매 판단을 해주세요.

        문서 내용:
        {text}

        현재 시장 데이터:
        - 현재가: {market_data['current_price']:,.0f}원
        - 전일 종가: {market_data['prev_closing_price']:,.0f}원
        - 변화율: {market_data['change_rate']:.2f}%

        다음 항목들을 분석해주세요:
        1. 시장 동향 요약
        2. 주요 위험 요소
        3. 투자 기회 요인
        4. 매매 추천 (매수/매도/관망)
        5. 추천 이유
        6. 위험도 평가 (0-10)

        JSON 형식으로 응답해주세요.
        """

        response = self.claude.messages.create(
            model="claude-3-sonnet-20240229",
            max_tokens=1000,
            temperature=0,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )

        return json.loads(response.content[0].text)

class MarketAnalysisAgent:
    """시장 분석 에이전트"""
    def __init__(self):
        self.market_data_history: List[Dict] = []

    def get_market_data(self) -> Dict:
        """실시간 시장 데이터 수집"""
        try:
            current_price = pyupbit.get_current_price("KRW-BTC")
            ohlcv = pyupbit.get_ohlcv("KRW-BTC", interval="day", count=1)
            
            data = {
                'current_price': float(current_price),
                'opening_price': float(ohlcv['open'][0]),
                'high_price': float(ohlcv['high'][0]),
                'low_price': float(ohlcv['low'][0]),
                'volume': float(ohlcv['volume'][0]),
                'prev_closing_price': float(ohlcv['close'].iloc[-1]),
                'change_rate': 0.0
            }
            
            data['change_rate'] = ((data['current_price'] - data['prev_closing_price']) 
                                 / data['prev_closing_price'] * 100)
            
            self.market_data_history.append(data)
            return data
        except Exception as e:
            print(f"시장 데이터 조회 실패: {e}")
            return None

    def analyze_market_trend(self) -> Dict:
        """시장 동향 분석"""
        if len(self.market_data_history) < 2:
            return {"trend": "unknown", "volatility": 0, "momentum": 0}

        recent_data = self.market_data_history[-10:]  # 최근 10개 데이터
        
        # 추세 분석
        price_changes = [d['change_rate'] for d in recent_data]
        avg_change = sum(price_changes) / len(price_changes)
        
        # 변동성 분석
        volatility = (max(d['high_price'] for d in recent_data) - 
                     min(d['low_price'] for d in recent_data)) / recent_data[-1]['current_price'] * 100
        
        # 모멘텀 계산
        momentum = sum(1 for c in price_changes if c > 0) / len(price_changes)

        return {
            "trend": "bullish" if avg_change > 0 else "bearish",
            "volatility": volatility,
            "momentum": momentum
        }

class DecisionMakingAgent:
    """의사결정 에이전트"""
    def __init__(self):
        self.decision_history: List[Dict] = []

    def make_decision(self, doc_analysis: Dict, market_analysis: Dict) -> Dict:
        """분석 결과를 바탕으로 매매 결정"""
        decision = {
            "action": "관망",  # 기본값
            "confidence": 0.0,
            "reason": "",
            "risk_level": 5.0
        }

        # 문서 분석 결과 반영
        doc_recommendation = doc_analysis.get('매매 추천', '관망')
        doc_risk = float(doc_analysis.get('위험도', 5))

        # 시장 분석 결과 반영
        market_trend = market_analysis['trend']
        market_volatility = market_analysis['volatility']
        market_momentum = market_analysis['momentum']

        # 매수 조건
        if (doc_recommendation == "매수" and 
            market_trend == "bullish" and 
            market_momentum > 0.6 and 
            doc_risk < 7):
            decision["action"] = "매수"
            decision["confidence"] = market_momentum
            decision["reason"] = "문서 분석과 시장 동향이 긍정적"

        # 매도 조건
        elif (doc_recommendation == "매도" or 
              (market_trend == "bearish" and market_volatility > 5)):
            decision["action"] = "매도"
            decision["confidence"] = 0.7
            decision["reason"] = "위험 신호 감지"

        decision["risk_level"] = (doc_risk + market_volatility/2) / 2
        self.decision_history.append(decision)
        return decision

class TradingExecutionAgent:
    """거래 실행 에이전트"""
    def __init__(self, initial_balance: float = 10000000):
        self.balance = initial_balance
        self.btc_held = 0.0
        self.trade_history: List[Dict] = []

    def execute_trade(self, decision: Dict, market_data: Dict) -> Dict:
        """매매 실행"""
        result = {
            "status": "실행 완료",
            "action": decision["action"],
            "quantity": 0.0,
            "price": market_data["current_price"],
            "timestamp": datetime.now()
        }

        if decision["action"] == "매수" and self.balance >= market_data["current_price"]:
            trade_amount = min(self.balance * 0.1, market_data["current_price"])
            quantity = trade_amount / market_data["current_price"]
            
            self.balance -= trade_amount
            self.btc_held += quantity
            result["quantity"] = quantity

        elif decision["action"] == "매도" and self.btc_held > 0:
            quantity = self.btc_held * 0.1  # 보유량의 10%
            trade_amount = quantity * market_data["current_price"]
            
            self.balance += trade_amount
            self.btc_held -= quantity
            result["quantity"] = quantity

        self.trade_history.append(result)
        return result

def main():
    # 에이전트 초기화
    doc_agent = DocumentAnalysisAgent(api_key="your_claude_api_key")
    market_agent = MarketAnalysisAgent()
    decision_agent = DecisionMakingAgent()
    execution_agent = TradingExecutionAgent()

    print("비트코인 AI 자동매매 시스템을 시작합니다.")
    print("=" * 50)

    try:
        while True:
            # 1. 문서 분석
            doc_text = doc_agent.parse_document("bitcoin_report.pdf")
            market_data = market_agent.get_market_data()
            
            if not doc_text or not market_data:
                print("데이터 수집 실패")
                continue

            doc_analysis = doc_agent.analyze_document(doc_text, market_data)
            
            # 2. 시장 분석
            market_analysis = market_agent.analyze_market_trend()
            
            # 3. 의사결정
            decision = decision_agent.make_decision(doc_analysis, market_analysis)
            
            # 4. 거래 실행
            result = execution_agent.execute_trade(decision, market_data)
            
            # 결과 출력
            print("\n" + "="*50)
            print(f"현재시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"현재가: {market_data['current_price']:,.0f}원")
            print(f"문서 분석: {doc_analysis['시장 동향 요약']}")
            print(f"시장 동향: {market_analysis['trend']}")
            print(f"위험도: {decision['risk_level']:.1f}/10")
            print(f"결정: {decision['action']} (신뢰도: {decision['confidence']:.2f})")
            print(f"실행결과: {result['status']}")
            print(f"보유 BTC: {execution_agent.btc_held:.8f}")
            print(f"보유 현금: {execution_agent.balance:,.0f}원")
            
            time.sleep(60)  # 1분 간격으로 실행

    except KeyboardInterrupt:
        print("\n프로그램을 종료합니다.")

if __name__ == "__main__":
    main()